Lattice LWE Implementation
=======================
---


Part of a project by Stanley Roberts on Lattice Cryptography  
This code is an implementation of *Regevs* public key cryptography mechanism using LWE

In [77]:
#### Imports ####

from sage.stats.distributions.discrete_gaussian_integer import DiscreteGaussianDistributionIntegerSampler
import copy
import math
import random

Module Info
----------


All default parameters are generated as suggested in Regev's paper but can be
smaller/different as long as they adhere to sufficient security constraints

see: 'A Framework to Select Parameters for Lattice-Based Cryptography'
and 'Better Key Sizes (and Attacks) for LWE-Based Encryption'

In [78]:
"""
Name
----
LWE_PKE

Description
-----------
This module implements a public key cryptography system using
LWE as detailed in Regev's paper On Lattices, Learning with Errors,
Random Linear Codes, and Cryptography

Contents
--------
publicKey : creates a public key
LWE : creates an LWE system
"""

"\nName\n----\nLWE_PKE\n\nDescription\n-----------\nThis module implements a public key cryptography system using\nLWE as detailed in Regev's paper On Lattices, Learning with Errors,\nRandom Linear Codes, and Cryptography\n\nContents\n--------\npublicKey : creates a public key\nLWE : creates an LWE system\n"

Helper Classes
---------------------

Smaller classes to assist in implementing the LWE system

In [79]:
class publicKey:
    """
    Creates a Public Key object
    
    Uses a matrix to create an LWE public key whose rows (excluding last element)
    are the vectors Ai and the last element in each row is the corresponding Bi.
    
    Parameters
    ----------
    
    pk : sagemath matrix
        matrix to use to construct public key.
    """
    def __init__(self, pk, q):
        self.pk = pk
        self.q = q
    
    def getA(self, i):
        """
        Get an A vector from the public key.
        
        Parameters
        ----------
        i : int
            vector index.
        
        Returns
        -------
        sagemath vector
            the appropriate A vector.
        """
        temp = list(self.pk.row(i))
        temp.pop()
        return vector(Integers(self.q), temp)
    
    def getB(self, i):
        """
        Get a B value from the public key.
        
        Parameters
        ----------
        i : int
            value index.
        
        Returns
        -------
        int
            the appropriate B value.
        """
        temp = list(self.pk.row(i))
        return temp.pop()
    
    def getSampleNo(self):
        return self.pk.nrows()
    
    def getModulus(self):
        return self.q
    
    def __str__(self):
        return self.pk.__str__() + " mod: " + self.q.__str__()
    
class cipherText:
    '''
    Generates an LWE ciphertext pair.
    
    Parameters
    ----------
    a : sagemath vector
        vector part of pair
    b : int
        integer part of pair
    
    '''
    def __init__(self, a, b):
        self.a = a
        self.b = b
    
    def __str__(self):
        return "[" + self.a.__str__() + ", " + self.b.__str__() + "]"
    

LWE Implementation
-----------------------------

Main implementation of LWE PKE system with encryption and decryption

In [82]:
class LWE:
    """
    Constructs an LWE enviroment.
    
    Constructs a new LWE enviroment based on given parameter and
    generates a public and private key for encryption and decryption.
    
    Parameters
    ----------
    n : int
        security parameter which defines dimension, defaults to 512
    q : int
        modulus, defaults to a random prime in the range of n^2 to 2n^2
    m : int
        desired number of samples for the public key, defaults to (n+1)log(q)
    x : callable object
        error distribution, defaults to a discrete gaussian with standard deviation 1/sqrt(n)log^2(n)
    """
    def __init__(self, n=512, q=None, m=None, x=None):
        self.n, self.q, self.m, self.x = n, q, m, x
        if q==None:
            self.q = random_prime(2*self.n^2, self.n^2) 
        if m==None:
            self.m = ((self.n+1)*self.q.log(prec=100)).integer_part()
        if x==None:
            alpha = (1/(sqrt(self.n)*log(self.n)^2))
            self.x = DiscreteGaussianDistributionIntegerSampler(alpha/sqrt(2*pi))
        
        self.VS = GF(self.q)^self.n #vector space, dimension n, modulus q
        self._s = self.VS.random_element() #secret key
        self._pk = self.__genPublicKey() #public key
    
    def getPublicKey(self):
        """
        Return public key associated with the LWE object
        
        Returns
        -------
        sagemath matrix
            the public key in matrix form [A, q]
        """
        return copy.deepcopy(self._pk)
    
    def __LWEsample(self):
        # gets a sample (a, b) where a is a randomly generated vector such that a ∈ VS
        # and b = <a, s> + e where e is sampled according to the error distribution
        
        a = self.VS.random_element()
        er = self.x()
        b = self._s.inner_product(a) + er
        return vector(Integers(self.q), list(a)+[b.lift()])

    def __genPublicKey(self):
        # generates a public key by combining m samples (from sampler) into a matrix form
        
        vector_list = [self.__LWEsample() for x in range(self.m)]
        return publicKey(matrix(vector_list), self.q)

    def enc(self, bit, key):
        """
        Encrypts a bit using the given key
        
        Parameters
        ----------
        bit : the binary bit to encrypt
        key : the public key to encrypt using
        
        Returns
        -------
        sagemath vector : vector composing one half of ciphertext pair (a)
        int : integer composing other half of pair (b)
        """
        
        sample_size = key.getSampleNo()
        q = key.getModulus()
        subset = Subsets(sample_size-1).random_element()
        a, b = 0, 0
        for i in subset:
            a += key.getA(i)
            b += key.getB(i)
        if bit==1:
            b += math.floor(q/2)
        return cipherText(a%q, b%q)
    
    def dec(self, pair):
        """
        Decrypts a bit which has been encrypted using the instance's public key
        
        Parameters
        ----------
        a : vector half of encryption pair
        b : integer half of encryption pair
        
        Returns
        -------
        int : decrypted bit
        
        """
        testval = lift(pair.b-pair.a.inner_product(self._s))
        compval = math.floor(self.q/2)
        
        if (min(0, compval, key = lambda x: abs(x-testval))==0): return 0
        return 1
    
    def encString(self, string):
        """
        Encrypts a string using the given key
        
        Parameters
        ----------
        string : plaintext string to encrypt
        key : public key to encrypt using
        
        Returns
        -------
        sagemath matrix : matrix whose rows correspond to a bit encryption pair
        """
        pass
    
    def decMatrix(self, a):
        """
        Decrypts a matrix whose rows each correspond to an encryption pair constructed using instance's public key
        
        Parameters
        ----------
        a : sagemath matrix to decrypt
        
        Returns
        -------
        string : plaintext string
        """
        pass

In [86]:
alice = LWE(n=150)
bob = LWE(n=200)

success, fail = 0, 0
tests = 200

for i in range (0, tests):
    message = random.randint(0, 1)
    cipher = alice.enc(message, bob.getPublicKey())
    plain = bob.dec(cipher)
    if (message == plain):
        success += 1
    else:
        fail += 1
        
print("Testing Alice -> Bob messages " + str(tests) + " times:\nSuccesses: " + str(success) + "\nFailures: " + str(fail) + "\nFailure rate: " + str(float((fail*100/tests))) + "%")

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
Testing Alice -> Bob messages 200 times:
Successes: 200
Failures: 0
Failure rate: 0.0%
